This notebook shows how to use [dvc](https://dvc.org/) [experiments](https://github.com/iterative/dvc/wiki/Experiments) in model development. This example uses the [MNIST](http://yann.lecun.com/exdb/mnist/) data of handwritten digits and builds a classification model to predict the digit (0-9) in each image. The model is built in [pytorch](https://pytorch.org/) as convolutional neural network with a simplified architecture, which should be able to quickly run on most computers.

### Getting started

To get started, clone this repository and navigate to it.

The only other prerequisite is [conda](https://docs.conda.io/projects/conda/en/latest/user-guide/install/). Once conda is installed, create a virtual environment from the existing `environment.yaml` file and activate it:

```bash
conda env create -f environment.yml
conda activate dvc
```

If you want to run this notebook directly, do so after activating the conda environment.

### Establishing the pipeline DAG

Before experimenting, a dvc pipeline must be established (see the docs if you are new to dvc). Review the contents of `dvc.yaml` below to see the pipeline.

In [1]:
%%bash
cat dvc.yaml

stages:
  download:
    cmd: python download.py
    deps:
    - download.py
    outs:
    - data/MNIST
  train:
    cmd: python train.py --model_path=model.pt --metrics_path=metrics.yaml
    deps:
    - data/MNIST
    - train.py
    params:
    - lr
    - weight_decay
    outs:
    - model.pt
    metrics:
    - metrics.yaml
  train_checkpoint:
    cmd: python train.py --model_path=model_checkpoint.pt --metrics_path=metrics_checkpoint.yaml --checkpoint=5
    deps:
    - data/MNIST
    - train.py
    params:
    - lr
    - weight_decay
    outs:
    - model_checkpoint.pt:
        checkpoint: true
    metrics:
    - metrics_checkpoint.yaml


The download stage gets the data using the `download.py` script. The train stage performs model training and evaluation on the downloaded data using the `train.py` script. The train stage uses the lr and weight_decay metrics defined in `params.yaml`. The model output is saved to `model.pt`, and the metrics are saved to `metrics.yaml`. The train_checkpoint stage is similar but saves output periodically.

Execute the pipeline to reproduce the train stage:

In [2]:
%%bash
dvc repro train

Running stage 'download' with command:
	python download.py
Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw
Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

Running stage 'train' with command:
	python train.py --model_path=model.pt --metrics_path=metrics.yaml
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add data/.gitignore dvc.lock
Use `dvc push` to send your updates to remote storage.


9920512it [00:02, 3784886.97it/s]                             
  0%|          | 0/28881 [00:00<?, ?it/s]
0it [00:00, ?it/s]
 98%|█████████▊| 1613824/1648877 [00:00<00:00, 2400675.96it/s]

0it [00:00, ?it/s]

  0%|          | 0/4542 [00:00<?, ?it/s]/home/dave/.conda/envs/dvc/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729002806/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
1654784it [00:00, 1984505.52it/s]                             
8192it [00:00, 33181.62it/s]        

**IMPORTANT:** Be sure to run the `git add` command above and also `git commit` before running experiments. Anytime you modify the pipeline, be sure to `dvc repro` and track changes with git before running experiments.

In [3]:
%%bash
git add data/.gitignore dvc.lock
git commit -m "download data"

[checkpoint 33caf3c] download data
 2 files changed, 33 insertions(+)
 create mode 100644 data/.gitignore
 create mode 100644 dvc.lock


### Run an experiment

Run an experiment with the default parameters defined in `params.yaml`.

In [4]:
%%bash
dvc exp run train

Stage 'download' didn't change, skipping
Stage 'train' didn't change, skipping
Reproduced experiment '33caf3c'.


Review the output of the run, including identifying hashes, metrics, and parameters:

In [5]:
%%bash
dvc exp show

┏━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Experiment ┃ Created  ┃    acc ┃   loss ┃ lr    ┃ weight_decay ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━┩
│ workspace  │ -        │ 0.2895 │ 2.2134 │ 0.001 │ 0            │
│ checkpoint │ 04:58 PM │ 0.2895 │ 2.2134 │ 0.001 │ 0            │
└────────────┴──────────┴────────┴────────┴───────┴──────────────┘


### Experimenting with different parameters

Experiments can be run and compared with different parameters.

In [6]:
%%bash
dvc exp run train --params weight_decay=0.1

Stage 'download' didn't change, skipping


ERROR: Failed to reproduce experiment '9cf47aa' - failed to reproduce 'dvc.yaml': restore() got an unexpected keyword argument 'checkpoint_resume'
